# ASL Action Recognition

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/03-emotion-analysis.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/03-emotion-analysis.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/03-emotion-analysis.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Start EVA server
We are reusing the start server notebook for launching the EVA server

In [ ]:
!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

In [ ]:
# An ASL video of the action computer
!wget -nc https://gatech.box.com/shared/static/q4bnbkcoplpazun25cjxke2801m0o33d.avi


### Adding the video file to EVADB for analysis

In [ ]:
cursor.execute('DROP TABLE ASL_ACTION')
response = cursor.fetch_all()
print(response)
cursor.execute('LOAD VIDEO "computer_asl.avi" INTO HAPPY')
response = cursor.fetch_all()
print(response)

### Visualize the video

In [1]:
from IPython.display import Video
Video("computer_asl.avi", height=450, width=800, embed=True)

### Create an user-defined function(UDF) for analyzing the frames

In [ ]:
cursor.execute("""CREATE UDF IF NOT EXISTS ASLActionRecognition
        INPUT  (Frame_Array NDARRAY UINT8(3, 16, 224, 224))
        OUTPUT (labels NDARRAY STR(ANYDIM))
        TYPE  Classification
        IMPL  '{}/udfs/asl_action_recognition.py
        """)
response = cursor.fetch_all()

### Run the UDF on the Video to find the ASL action

In [ ]:
cursor.execute("""SELECT FIRST(id), ASLActionRecognition(SEGMENT(data)) FROM Asl_actions
                        SAMPLE 5 GROUP BY '16f';""")
response = cursor.fetch_all()
print(response)